In [1]:
using StatsBase;
using ProgressBars;
using BenchmarkTools;
using Plots;
using Random;

## Helper Structs

In [2]:
mutable struct Location
    n::Int
    x::Float64
    y::Float64
    demand::Int
    ready_time::Int
    due_time::Int
    service_time::Int
    is_depot::Bool
    
    function Location(n, x, y, demand, ready_time, due_time, service_time, is_depot)
        return new(n, x, y, demand, ready_time, due_time, service_time, is_depot)
    end
end

In [3]:
function dist2(a::Location, b::Location)
    return ((a.x - b.x) ^ 2 + (a.y - b.y) ^ 2) ^ 0.5
end

dist2 (generic function with 1 method)

In [4]:
struct Problem
    name::String
    n_locations::Int
    n_customers::Int
    n_vehicles::Int
    n_depots::Int
    locations::Vector{Location}
    vehicle_capacity::Int
    edges::Array{Float64, 2}
    
    function Problem(name, n_customers, n_vehicles, locations, vehicle_capacity)
        n_depots = n_vehicles + 1
        n_locations = n_customers + n_depots
        
        edges = Array{Float64, 2}(undef, n_locations, n_locations)
        
        xf = 10000
        for i in 1:n_locations
            locations[i].n = i
            for j in 1:n_locations
                edges[i,j] = dist2(locations[i], locations[j])
            end
            
            edges[i, i] = xf # distance to itself is high
        end
        
        # distance between depots is high
        for i in 101:n_locations
            for j in 101:n_locations
                edges[i,j] = xf
            end
        end
        return new(name, n_locations, n_customers, n_vehicles, n_depots, locations, vehicle_capacity, edges)
    end
end

In [5]:
mutable struct Route
    locations::Vector{Int}
    distance::Float64
    n_customers::Int
    is_valid::Bool
    
    function Route(locations, p)
        n_customers = size(locations)[1] - 2
        
        if n_customers == 0
            return new(locations, 0, 0, true)
        end
        
        distance = 0
        is_valid = true
        time = 0
        load = 0
        for i in 1:n_customers+1
            this_stop = p.locations[locations[i]]
            next_stop = p.locations[locations[i+1]]
            
            dtn = p.edges[this_stop.n, next_stop.n]
            
            if time + dtn <= next_stop.due_time && load + next_stop.demand <= p.vehicle_capacity
                time = max(time + dtn, next_stop.ready_time)
                time += next_stop.service_time
                distance += dtn
                load += next_stop.demand
            else
                is_valid = false
                dist = Inf
                break
            end            
        end
        return new(locations, distance, n_customers, is_valid)
    end
end

In [6]:
mutable struct Solution
    tour::Vector{Int}
    routes::Vector{Route}
    distance::Float64
    n_locations::Int
    n_customers::Int
    n_vehicles::Int
    n_depots::Int
    is_valid::Bool
    
    function Solution(tour, p)
        
        routes = []
        locations = [tour[1]]
        n_locations = size(tour)[1]
        distance = 0
        n_vehicles = 0
        n_customers = 0
        is_valid = true
        
        for x in tour[2:n_locations]
            
            if p.locations[x].is_depot
                push!(locations, x)
                route = Route(locations, p)
                push!(routes, route)
                if !route.is_valid
                    is_valid = false
                    dist = Inf
                    break
                end
                distance += route.distance
                n_vehicles += 1
                locations = [x]
            else
                n_customers += 1
                push!(locations, x)
            end
        end
        
        n_depots = n_vehicles + 1
        if is_valid
            @assert n_depots + n_customers == n_locations
        end
        
        if n_customers != 100 || n_vehicles > p.n_vehicles
            is_valid = false
        end
        
        return new(tour, routes, distance, n_locations, n_customers, n_vehicles, n_depots, is_valid)
    end
end

In [7]:
function update(s::Solution, p::Problem)
    tour = [101]
    nd = 102
    for route in s.routes
        if size(route.locations)[1] == 2
           continue 
        end
        for loc in route.locations[2:size(route.locations)[1]]
            if p.locations[loc].is_depot
                push!(tour, nd)
                nd += 1
            else
                push!(tour, loc)
            end
        end
    end
    return Solution(tour, p)
end

update (generic function with 1 method)

In [8]:
function get_problem(filename)
    
    open("data/$filename.txt") do file

        i = 1
        name = ""
        depot = 0
        n_customers = 0
        n_vehicles, vehicle_capacity = 0, 0
        skip = [2,3,4,6,7,8,9]
        locations = []

        for ln in eachline(file)
            if ! (i in skip)
                if i == 1
                    name = split(ln)[1]
                elseif i == 5
                    n_vehicles, vehicle_capacity = [parse(Int64, a) for a in split(ln)]
                else
                    n, x, y, demand, ready_time, due_date, service_time = [parse(Int64, a) for a in split(ln)]
                    is_depot = (n == 0 ? true : false)
                    is_serviced = false
                    if n != 0
                        n_customers += 1
                        push!(locations, Location(n, x, y, demand, ready_time, due_date, service_time, is_depot))
                    else
                        depot = Location(n, x, y, demand, ready_time, due_date, service_time, is_depot)
                    end
                end
            end
            i+=1
        end

        for i in 1:n_vehicles+1
            push!(locations, deepcopy(depot))
        end

        return Problem(name, n_customers, n_vehicles, locations, vehicle_capacity)
    end
end

get_problem (generic function with 1 method)

In [9]:
p = get_problem("r103")
p.name

"R103"

In [10]:
function plot_problem(p)
    x_l = [loc.x for loc in p.locations]
    y_l = [loc.y for loc in p.locations]
    scatter(x_l, y_l, legend = false)
end 

# plot_problem(p);

In [11]:
tour = [101,60,45,83,5,99,6,101,71,65,78,34,35,81,77,28,101,2,22,75,56,4,25,54,101,7,19,11,8,46,47,48,82,18,89,101,94,96,95,97,87,13,101,27,69,30,9,66,20,51,1,101,42,43,15,57,41,74,72,73,21,58,101,40,53,12,68,80,101,50,33,76,79,10,31,101,36,64,49,63,90,32,70,101,92,98,14,44,38,86,16,61,85,91,100,37,101,26,39,23,67,55,24,29,3,101,52,62,88,84,17,93,59,101]
sol = Solution(tour, p)
sol.distance, sol.is_valid, sol.n_vehicles

(1292.6755102213538, true, 13)

## Implementation of Solution

### Initial solution

In [12]:
function initial_solution(p::Problem, max_vehicles=nothing)
    tour = [101]
    vis = zeros(Bool, p.n_locations)
    vis[101] = true
    nd = 102
    time = 0
    load = 0
    vehicles_used = 0
    
    while false in vis[1:100]
        lv = last(tour) # last visited
        added = false
        for nv in sortperm(p.edges[lv, 1:100]) # next visit
            next_stop = p.locations[nv]
            next_depot = p.locations[nd]
            
            dtn = p.edges[lv, nv] # distance to next
            dtd = p.edges[nv, nd] # distance to depot from next location
            
            if (!vis[nv] && time + dtn <= next_stop.due_time && load + next_stop.demand <= p.vehicle_capacity 
                    && max(time + dtn, next_stop.ready_time) + next_stop.service_time + dtd <= next_depot.due_time)
                push!(tour, nv)
                vis[nv] = true
                time = max(time + dtn, next_stop.ready_time) + next_stop.service_time
                load += next_stop.demand
                added = true
                break
            end
        end
        if !added                
            push!(tour, nd)
            vis[nd] = true
            nd += 1
            vehicles_used += 1
            if max_vehicles != nothing && vehicles_used == max_vehicles
               break 
            end
            time = 0
            load = 0
        end
    end
    
    if p.locations[last(tour)].is_depot == false
        push!(tour, nd)
    end
    return Solution(tour, p)
end

initial_solution (generic function with 2 methods)

In [13]:
# fisw = initial_solution(p)
# @info fisw.tour, fisw.n_vehicles, fisw.n_customers, fisw.is_valid

In [14]:
# fisv = initial_solution(p, 15)
# @info fisv.tour, fisv.n_vehicles, fisv.n_customers, fisv.is_valid

## Local Search

In [15]:
function two_opt(a, i, j, n)
    if i == 1
        return vcat(reverse(a[i+1:j]), [a[i]], a[j+1:n])
    end
    return vcat(a[1:i-1], reverse(a[i:j]), a[j+1:n])
end

function cross(a, b, i, j)
    n = size(a)[1]
    m = size(b)[1]
    return vcat(a[1:i-1], b[j:m]), vcat(b[1:j-1], a[i:n])
end

function insertion(a, b, i, j)
    n = size(a)[1]
    m = size(b)[1]
    if n == 0
        return a, b
    end
    while i > n
        i -= n
    end
    return vcat(a[1:i-1], a[i+1:n]), vcat(b[1:j-1], [a[i]], b[j:m])
end

function swap(a, b, i, j)
    n = size(a)[1]
    m = size(b)[1]
    if i > n || j > m
        return a, b
    end
    a = copy(a)
    b = copy(b)
    a[i], b[j] = b[j], a[i]
    return a, b
end

swap (generic function with 1 method)

In [16]:
function local_search(p::Problem, s::Solution)
    
    can_impr = true
    n_routes = size(s.routes)[1]
    
    while can_impr
        can_impr = false
        
        for i = 1:n_routes, j=i+1:n_routes

            for k = 1:s.routes[i].n_customers, l=1:s.routes[j].n_customers
                
                r1s, r1e = first(s.routes[i].locations), last(s.routes[i].locations)
                r2s, r2e = first(s.routes[j].locations), last(s.routes[j].locations)
                
                c1, c2 =  insertion(s.routes[i].locations[2:size(s.routes[i].locations)[1]-1],
                            s.routes[j].locations[2:size(s.routes[j].locations)[1]-1],
                            k, l)
                c3, c4 =  swap(s.routes[i].locations[2:size(s.routes[i].locations)[1]-1],
                            s.routes[j].locations[2:size(s.routes[j].locations)[1]-1],
                            k, l)
                c5, c6 =  cross(s.routes[i].locations[2:size(s.routes[i].locations)[1]-1],
                            s.routes[j].locations[2:size(s.routes[j].locations)[1]-1],
                            k, l)

                n1, n2 = Route(vcat([r1s], c1, [r1e]), p), Route(vcat([r2s], c2, [r2e]), p)
                n3, n4 = Route(vcat([r1s], c3, [r1e]), p), Route(vcat([r2s], c4, [r2e]), p)
                n5, n6 = Route(vcat([r1s], c5, [r1e]), p), Route(vcat([r2s], c6, [r2e]), p)

                if n1.is_valid && n2.is_valid
                    if n1.distance + n2.distance < s.routes[i].distance + s.routes[j].distance
                        s.routes[i] = n1
                        s.routes[j] = n2
                        can_impr = true
                        break
                    end
                end
                
                if n3.is_valid && n4.is_valid
                    if n3.distance + n4.distance < s.routes[i].distance + s.routes[j].distance
                        s.routes[i] = n3
                        s.routes[j] = n4
                        can_impr = true
                        break
                    end
                end
                
                if n5.is_valid && n6.is_valid
                    if n5.distance + n6.distance < s.routes[i].distance + s.routes[j].distance
                        s.routes[i] = n5
                        s.routes[j] = n6
                        can_impr = true
                        break
                    end
                end
            end
        end
    end
 
    s = update(s, p)
    return s
end

local_search (generic function with 1 method)

In [17]:
# @time ls = local_search(p, fisw)
# @info ls

## Creating new ant

In [18]:
function new_active_ant(dolocal, inc, n_vehicles, p, tau, tau0, beta, rho)

    n_depots = n_vehicles + 1
    n_locations = p.n_customers + n_depots
    tour = [100 + rand(1:n_depots)]
    time = 0
    load = 0
    vehicles_used = 0

    vis = zeros(Bool, n_locations)
    vis[tour[1]] = true
    eta = fill(0.00001, n_locations)

    feasible_node = true

    while feasible_node

        feasible_node = false
        lv = last(tour) # last vertex
        
        for nv in 1:n_locations
            next_stop = p.locations[nv]
            some_depot = p.locations[101]
            
            dtn = p.edges[lv, nv] # distance to next
            dtd = p.edges[nv, 101] # distance to depot from next location
            
            if (!vis[nv] && !next_stop.is_depot # if next location is a customer
                    && time + dtn <= next_stop.due_time 
                    && load + next_stop.demand <= p.vehicle_capacity 
                    && max(time + dtn, next_stop.ready_time) + next_stop.service_time + dtd <= some_depot.due_time)
                
                feasible_node = true
                deliv_time = max(time + dtn, next_stop.ready_time)
                delta_time = deliv_time - time
                distance = delta_time * (next_stop.due_time - time)
                distance = max(1, distance + inc[nv])
                eta[nv] = 1/ distance
                
            elseif (!vis[nv] && next_stop.is_depot  # if next location is a depot
                    && time + dtn <= next_stop.due_time)
                
                feasible_node = true
                deliv_time = max(time + dtn, next_stop.ready_time)
                delta_time = deliv_time - time
                distance = delta_time * (next_stop.due_time - time)
                distance = max(1, distance - inc[nv])
                eta[nv] = 1/ distance
                
            else
                eta[nv] = 0
            end
            
        end

        if !feasible_node
            break
        end

        w = (tau[lv,1:n_locations]) .* (eta .^ beta)
        w = w / sum(w)
        
        # next = np.random.choice(n_locations + n_depots, p=probs)
        q = rand()
        if q < 0.9
            nv = argmax(w)
        else
            w[w .!= 0.0] .= 1
            w = w / sum(w)
            nv = sample(1:n_locations, ProbabilityWeights(w))
        end
        
#         @info "probs" w nv tour

        next_stop = p.locations[nv]
        dtn = p.edges[lv, nv] # distance to next
        push!(tour, nv)
        vis[nv] = true
        if next_stop.is_depot
            time = 0
            load = 0
            vehicles_used += 1
            if vehicles_used == n_vehicles
                break
            end
        else
            time = max(time + dtn, next_stop.ready_time) + next_stop.service_time
            load += next_stop.demand
        end
        
        tau[lv,nv] = (1 - rho) * tau[lv,nv] + rho * tau0
    end
    
    # there still may remain some non inserted nodes
    for nv in 1:n_locations
        if !vis[nv]
            for g in 2:size(tour)[1]+1
                tc = copy(tour)
                insert!(tc, g, nv)
                if Solution(tc, p).is_valid
                    tour = tc
                    vis[nv] = true
                    break
                end
            end
        end
    end
    
    sol = Solution(tour, p)
    return dolocal ? local_search(p, sol) : sol
end

new_active_ant (generic function with 1 method)

In [19]:
# @benchmark new_active_ant(false, fill(0, p.n_locations + p.n_depots), 20, p, 
#                         fill(10/sum(p.edges), size(p.edges)), 10/sum(p.edges), 1, 0.1)

In [20]:
# @benchmark new_active_ant(true, fill(0, p.n_locations + p.n_depots), 20, p, 
#                         fill(10/sum(p.edges), size(p.edges)), 10/sum(p.edges), 1, 0.1)

## ACO Vehicle function

In [21]:
function aco_vehicle(n_vehicles, p, n_ants, beta, rho)
    global psi_gb

    tau0 = 10/sum(p.edges)
    tau = fill(tau0, size(p.edges))

    # initial solution, may be infeasible
    psi_vei = initial_solution(p, n_vehicles)

    inc = fill(0, p.n_locations + p.n_depots)
    
    for t in 1:100
        ants = []

        for i in 1:n_ants
            push!(ants, new_active_ant(false, inc, n_vehicles, p, tau, tau0, beta, rho))
            for x in last(ants).tour
                inc[x] += 1 # CHANGE
            end
        end
 
        for i in 1:n_ants
            if ants[i].n_customers > psi_vei.n_customers
                psi_vei = deepcopy(ants[i])
                inc = fill(0, p.n_locations + p.n_depots)
                if ants[i].is_valid && ants[i].distance < psi_gb.distance
                    psi_gb = deepcopy(ants[i])
                end
            end
        end
            
        # updating tau values
        tau = (1-rho) * tau
        for j in 1:psi_vei.n_locations-1   
            lv, nv = psi_vei.tour[j:j+1]
            tau[lv,nv] += 1/psi_vei.distance
            tau[lv,nv] += 1/psi_vei.distance
        end
        
        for j in 1:psi_gb.n_locations-1   
            lv, nv = psi_gb.tour[j:j+1]
            if lv < size(tau)[1] && nv < size(tau)[2]
                tau[lv,nv] += 1/psi_gb.distance
                tau[lv,nv] += 1/psi_gb.distance
            end
        end
    end

    return psi_vei
end

aco_vehicle (generic function with 1 method)

In [22]:
# psi_gb = initial_solution(p)
# acov = aco_vehicle(16, p, 10, 1, 0.1)

## ACO Time function

In [23]:
function aco_time(n_vehicles, p, n_ants, beta, rho)
    global psi_gb

    tau0 = 10/sum(p.edges)
    tau = fill(tau0, size(p.edges))

    # initial solution, may be infeasible
    psi_vei = initial_solution(p, n_vehicles)

    inc = fill(0, p.n_locations + p.n_depots)
    
    for t in 1:10
        ants = []

        for i in 1:n_ants
            push!(ants, new_active_ant(true, inc, n_vehicles, p, tau, tau0, beta, rho))
        end
 
        for i in 1:n_ants
            if ants[i].is_valid && ants[i].distance < psi_gb.distance
                psi_gb = deepcopy(ants[i])
            end
        end
            
        # updating tau values
        tau = (1 - rho) * tau
        
        for j in 1:psi_gb.n_locations-1   
            lv, nv = psi_gb.tour[j:j+1]
            if lv < size(tau)[1] && nv < size(tau)[2]
                tau[lv,nv] += 1/psi_gb.distance
                tau[lv,nv] += 1/psi_gb.distance
            end
        end
    end

    return psi_gb
end

aco_time (generic function with 1 method)

In [24]:
# psi_gb = initial_solution(p)
# acov = aco_time(17, p, 10, 1, 0.1)

## Higher level multiple ACO function

In [25]:
function maco_vrptw(p, n_ants, beta, rho)
    global psi_gb
    
    veh = psi_gb.n_vehicles
    for i in 1:30
        
        psi_v = aco_vehicle(veh - 1, p, n_ants, beta, rho)
        @info "aco_vehicle" veh-1 psi_v.n_customers psi_v.distance 
        if psi_v.n_vehicles <= veh && psi_v.is_valid
            psi_gb = deepcopy(psi_v)
            veh = psi_gb.n_vehicles
            continue
        end
        psi_t = aco_time(veh, p, n_ants, beta, rho)
        @info "aco_time" veh psi_t.n_customers psi_t.distance 
        
    end
    return local_search(p, psi_gb)
end

maco_vrptw (generic function with 1 method)

In [26]:
psi_gb = initial_solution(p)
best = maco_vrptw(p, 10, 1, 0.1)

0.0%┣                                           ┫ 0/30 [00:00<00:-2, -20.0 it/s]
3.3%┣█▍                                         ┫ 1/30 [00:02<Inf:Inf, 0.0 it/s]


┌ Info: aco_vehicle
│   veh - 1 = 24
│   psi_v.n_customers = 100
│   psi_v.distance = 1929.3962358544975
└ @ Main In[25]:8


6.7%┣███                                          ┫ 2/30 [00:02<00:54, 0.5 it/s]


┌ Info: aco_vehicle
│   veh - 1 = 23
│   psi_v.n_customers = 100
│   psi_v.distance = 1888.7768806911745
└ @ Main In[25]:8


10.0%┣████▍                                       ┫ 3/30 [00:02<00:30, 0.9 it/s]

...

96.7%┣█████████████████████████████████████████▋ ┫ 29/30 [04:22<00:09, 0.1 it/s]


┌ Info: aco_time
│   veh = 15
│   psi_t.n_customers = 100
│   psi_t.distance = 1305.2031324087475
└ @ Main In[25]:15
┌ Info: aco_vehicle
│   veh - 1 = 14
│   psi_v.n_customers = 96
│   psi_v.distance = 1645.8714936197407
└ @ Main In[25]:8


100.0%┣██████████████████████████████████████████┫ 30/30 [04:34<00:00, 0.1 it/s]
100.0%┣██████████████████████████████████████████┫ 30/30 [04:34<00:00, 0.1 it/s]



┌ Info: aco_time
│   veh = 15
│   psi_t.n_customers = 100
│   psi_t.distance = 1305.2031324087475
└ @ Main In[25]:15


Solution([101, 40, 53, 102, 36, 64, 49, 19, 7, 103  …  100, 115, 28, 76, 79, 3, 68, 29, 24, 116], Route[Route([101, 40, 53, 102], 22.360679774997898, 2, true), Route([102, 36, 64, 49, 19, 7, 103], 119.96447761124713, 5, true), Route([103, 83, 45, 84, 17, 61, 93, 104], 79.31036984397787, 6, true), Route([104, 71, 65, 9, 66, 20, 51, 105], 126.5916452391877, 6, true), Route([105, 21, 72, 22, 74, 73, 106], 56.58347306532235, 5, true), Route([106, 27, 69, 30, 10, 107], 61.116871521370776, 4, true), Route([107, 50, 33, 81, 78, 34, 35, 77, 12, 108], 96.84192958915898, 8, true), Route([108, 26, 39, 23, 67, 55, 25, 109], 110.79907403480814, 6, true), Route([109, 1, 31, 62, 11, 63, 90, 32, 70, 110], 94.2081541322224, 8, true), Route([110, 52, 88, 8, 46, 47, 48, 82, 18, 89, 111], 95.36149850738124, 9, true), Route([111, 96, 98, 37, 59, 99, 6, 94, 112], 51.35292509685719, 7, true), Route([112, 2, 43, 15, 57, 41, 75, 56, 4, 54, 80, 113], 120.9880485211177, 10, true), Route([113, 92, 42, 95, 97, 87,

In [27]:
print(best.tour)

[101, 40, 53, 102, 36, 64, 49, 19, 7, 103, 83, 45, 84, 17, 61, 93, 104, 71, 65, 9, 66, 20, 51, 105, 21, 72, 22, 74, 73, 106, 27, 69, 30, 10, 107, 50, 33, 81, 78, 34, 35, 77, 12, 108, 26, 39, 23, 67, 55, 25, 109, 1, 31, 62, 11, 63, 90, 32, 70, 110, 52, 88, 8, 46, 47, 48, 82, 18, 89, 111, 96, 98, 37, 59, 99, 6, 94, 112, 2, 43, 15, 57, 41, 75, 56, 4, 54, 80, 113, 92, 42, 95, 97, 87, 13, 58, 114, 60, 5, 16, 44, 38, 14, 86, 85, 91, 100, 115, 28, 76, 79, 3, 68, 29, 24, 116]

## Solutions

r103.txt =>

Solution([101, 31, 62, 11, 19, 49, 47, 46, 48, 7, 102, 28, 50, 33, 3, 76, 79, 68, 29, 24, 103, 1, 30, 9, 71, 66, 20, 51, 104, 65, 34, 78, 35, 81, 77, 105, 21, 73, 22, 72, 74, 57, 2, 58, 106, 42, 43, 15, 41, 75, 56, 4, 54, 80, 12, 107, 40, 53, 108, 26, 39, 23, 67, 55, 25, 109, 87, 97, 59, 93, 110, 27, 69, 88, 8, 82, 111, 36, 64, 63, 90, 10, 32, 70, 112, 89, 45, 83, 84, 17, 61, 113, 52, 18, 60, 5, 96, 99, 6, 94, 95, 13, 114, 92, 85, 98, 44, 38, 14, 86, 16, 91, 100, 37, 115], 1303.0497514569026, 115, 100, 14, 15, true)

Solution([101, 76, 79, 68, 3, 77, 102, 50, 33, 78, 34, 35, 51, 103, 96, 99, 6, 104, 39, 23, 67, 55, 25, 54, 105, 92, 44, 38, 14, 91, 98, 37, 100, 106, 71, 65, 66, 20, 70, 1, 107, 94, 95, 97, 87, 13, 58, 108, 27, 69, 30, 9, 81, 29, 24, 80, 109, 40, 53, 110, 2, 22, 72, 73, 74, 21, 26, 111, 31, 88, 8, 46, 17, 60, 5, 112, 42, 43, 15, 57, 41, 75, 56, 4, 12, 28, 113, 52, 7, 62, 11, 19, 10, 32, 90, 63, 114, 36, 64, 49, 47, 48, 82, 18, 89, 115, 83, 45, 84, 61, 86, 16, 85, 93, 59, 116], 1291.4342990352254, 116, 100, 15, 16, true)